<a href="https://colab.research.google.com/github/ClaFlorez/Machine_Learning_Simplifie/blob/main/8_4_Evaluation_complete_du_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Évaluation complète du clustering
from sklearn.cluster import KMeans, DBSCAN
from sklearn.datasets import make_blobs
from sklearn.metrics import silhouette_score, silhouette_samples, calinski_harabasz_score, davies_bouldin_score
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Créer plusieurs datasets avec différentes qualités de clustering
print("Évaluation de la qualité du clustering")
print("Comparaison sur différents types de données")
print("=" * 60)

np.random.seed(42)

# Dataset 1: Clusters bien séparés (facile)
X_easy, y_easy = make_blobs(n_samples=300, centers=4, n_features=2,
                           cluster_std=1.0, random_state=42)

# Dataset 2: Clusters proches (difficile)
X_hard, y_hard = make_blobs(n_samples=300, centers=4, n_features=2,
                           cluster_std=2.5, random_state=42)

# Dataset 3: Clusters avec densités différentes
X_varied = []
y_varied = []
centers_varied = [[0, 0], [8, 8], [0, 8], [8, 0]]
stds_varied = [0.5, 1.5, 1.0, 2.0]  # Densités différentes

for i, (center, std) in enumerate(zip(centers_varied, stds_varied)):
    n_points = [100, 50, 75, 75][i]  # Tailles différentes aussi
    cluster_points = np.random.normal(center, std, (n_points, 2))
    X_varied.extend(cluster_points)
    y_varied.extend([i] * n_points)

X_varied = np.array(X_varied)
y_varied = np.array(y_varied)

datasets = {
    'Facile (bien séparés)': (X_easy, y_easy),
    'Difficile (proches)': (X_hard, y_hard),
    'Densités variées': (X_varied, y_varied)
}

print(f"Trois datasets de test créés:")
for name, (X, y) in datasets.items():
    print(f"  {name}: {len(X)} points, {len(np.unique(y))} clusters vrais")

# Tester K-Means avec différents K sur chaque dataset
fig, axes = plt.subplots(3, 4, figsize=(20, 15))

metrics_results = {}

for dataset_idx, (dataset_name, (X, y_true)) in enumerate(datasets.items()):
    print(f"\nAnalyse du dataset: {dataset_name}")
    print("=" * 50)

    # Standardiser
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Tester différents nombres de clusters
    k_range = range(2, 8)
    silhouette_scores = []
    calinski_scores = []
    davies_bouldin_scores = []

    for k in k_range:
        # Appliquer K-Means
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        clusters = kmeans.fit_predict(X_scaled)

        # Calculer les métriques
        sil_score = silhouette_score(X_scaled, clusters)
        calinski_score = calinski_harabasz_score(X_scaled, clusters)
        db_score = davies_bouldin_score(X_scaled, clusters)

        silhouette_scores.append(sil_score)
        calinski_scores.append(calinski_score)
        davies_bouldin_scores.append(db_score)

        print(f"K={k}: Silhouette={sil_score:.3f}, Calinski={calinski_score:.1f}, Davies-Bouldin={db_score:.3f}")

    # Stocker les résultats
    metrics_results[dataset_name] = {
        'silhouette': silhouette_scores,
        'calinski': calinski_scores,
        'davies_bouldin': davies_bouldin_scores,
        'k_range': list(k_range)
    }

    # Visualiser les données originales
    axes[dataset_idx, 0].scatter(X[:, 0], X[:, 1], c=y_true, cmap='tab10', alpha=0.7)
    axes[dataset_idx, 0].set_title(f'{dataset_name}\n(Vraies classes)', fontweight='bold')

    # Trouver le K optimal selon chaque métrique
    best_k_sil = k_range[np.argmax(silhouette_scores)]
    best_k_cal = k_range[np.argmax(calinski_scores)]
    best_k_db = k_range[np.argmin(davies_bouldin_scores)]  # Davies-Bouldin: plus bas = mieux

    print(f"K optimal selon Silhouette: {best_k_sil}")
    print(f"K optimal selon Calinski-Harabasz: {best_k_cal}")
    print(f"K optimal selon Davies-Bouldin: {best_k_db}")

    # Visualiser le clustering avec K optimal (silhouette)
    kmeans_optimal = KMeans(n_clusters=best_k_sil, random_state=42, n_init=10)
    clusters_optimal = kmeans_optimal.fit_predict(X_scaled)

    axes[dataset_idx, 1].scatter(X[:, 0], X[:, 1], c=clusters_optimal, cmap='tab10', alpha=0.7)
    axes[dataset_idx, 1].scatter(kmeans_optimal.cluster_centers_[:, 0], kmeans_optimal.cluster_centers_[:, 1],
                                c='red', marker='x', s=200, linewidths=3)
    axes[dataset_idx, 1].set_title(f'K-Means (K={best_k_sil})\nSilhouette={silhouette_scores[best_k_sil-2]:.3f}',
                                  fontweight='bold')

# Graphiques de métriques
for dataset_idx, (dataset_name, metrics) in enumerate(metrics_results.items()):
    k_range = metrics['k_range']

    # Silhouette scores
    axes[dataset_idx, 2].plot(k_range, metrics['silhouette'], 'o-', linewidth=2, markersize=8)
    axes[dataset_idx, 2].set_title(f'Silhouette Score\n{dataset_name}', fontweight='bold')
    axes[dataset_idx, 2].set_xlabel('Nombre de Clusters (K)')
    axes[dataset_idx, 2].set_ylabel('Silhouette Score')
    axes[dataset_idx, 2].grid(True, alpha=0.3)

    # Marquer le maximum
    best_idx = np.argmax(metrics['silhouette'])
    best_k = k_range[best_idx]
    best_score = metrics['silhouette'][best_idx]
    axes[dataset_idx, 2].axvline(best_k, color='red', linestyle='--', alpha=0.7)
    axes[dataset_idx, 2].text(best_k + 0.1, best_score, f'Max: {best_score:.3f}',
                             fontweight='bold')

    # Davies-Bouldin (plus bas = mieux)
    axes[dataset_idx, 3].plot(k_range, metrics['davies_bouldin'], 's-', linewidth=2, markersize=8, color='red')
    axes[dataset_idx, 3].set_title(f'Davies-Bouldin Index\n{dataset_name}', fontweight='bold')
    axes[dataset_idx, 3].set_xlabel('Nombre de Clusters (K)')
    axes[dataset_idx, 3].set_ylabel('Davies-Bouldin Index')
    axes[dataset_idx, 3].grid(True, alpha=0.3)

    # Marquer le minimum
    best_idx_db = np.argmin(metrics['davies_bouldin'])
    best_k_db = k_range[best_idx_db]
    best_score_db = metrics['davies_bouldin'][best_idx_db]
    axes[dataset_idx, 3].axvline(best_k_db, color='red', linestyle='--', alpha=0.7)
    axes[dataset_idx, 3].text(best_k_db + 0.1, best_score_db, f'Min: {best_score_db:.3f}',
                             fontweight='bold')

plt.tight_layout()
plt.show()

# Analyser la cohérence entre métriques
print(f"\nAnalyse de cohérence entre métriques:")
print("=" * 60)

for dataset_name, metrics in metrics_results.items():
    k_range = metrics['k_range']

    best_k_sil = k_range[np.argmax(metrics['silhouette'])]
    best_k_cal = k_range[np.argmax(metrics['calinski'])]
    best_k_db = k_range[np.argmin(metrics['davies_bouldin'])]

    print(f"\n{dataset_name}:")
    print(f"  Silhouette recommande K={best_k_sil}")
    print(f"  Calinski-Harabasz recommande K={best_k_cal}")
    print(f"  Davies-Bouldin recommande K={best_k_db}")

    # Vérifier la cohérence
    recommendations = [best_k_sil, best_k_cal, best_k_db]
    if len(set(recommendations)) == 1:
        print(f"  ✅ CONSENSUS: Toutes les métriques recommandent K={recommendations[0]}")
    elif len(set(recommendations)) == 2:
        print(f"  ⚠️ ACCORD PARTIEL: Pas de consensus complet")
    else:
        print(f"  ❌ DÉSACCORD: Métriques contradictoires")

# Guide d'interprétation des métriques
print(f"\nGuide d'interprétation des métriques:")
print("=" * 60)

print("SILHOUETTE SCORE (-1 à +1):")
print("  > 0.7  : Excellente séparation")
print("  0.5-0.7: Bonne séparation")
print("  0.3-0.5: Séparation correcte")
print("  < 0.3  : Séparation faible")
print("  < 0   : Clustering incorrect")

print("\nCALINSKI-HARABASZ INDEX (plus élevé = mieux):")
print("  > 1000 : Très bons clusters")
print("  500-1000: Bons clusters")
print("  100-500: Clusters corrects")
print("  < 100  : Clusters faibles")

print("\nDAVIES-BOULDIN INDEX (plus bas = mieux):")
print("  < 0.5  : Excellents clusters")
print("  0.5-1.0: Bons clusters")
print("  1.0-1.5: Clusters corrects")
print("  > 1.5  : Clusters faibles")

# Analyse des silhouettes individuelles
print(f"\nAnalyse des silhouettes individuelles:")
print("=" * 50)

# Prendre le premier dataset pour l'analyse détaillée
X_example, y_example = datasets['Facile (bien séparés)']
scaler_example = StandardScaler()
X_example_scaled = scaler_example.fit_transform(X_example)

# Appliquer K-Means avec K=4
kmeans_example = KMeans(n_clusters=4, random_state=42, n_init=10)
clusters_example = kmeans_example.fit_predict(X_example_scaled)

# Calculer les silhouettes individuelles
silhouette_individual = silhouette_samples(X_example_scaled, clusters_example)

# Visualiser les silhouettes par cluster
plt.figure(figsize=(15, 10))

plt.subplot(2, 2, 1)
y_lower = 10
colors = plt.cm.tab10(np.linspace(0, 1, 4))

for i in range(4):
    cluster_silhouettes = silhouette_individual[clusters_example == i]
    cluster_silhouettes.sort()

    size_cluster_i = cluster_silhouettes.shape[0]
    y_upper = y_lower + size_cluster_i

    plt.fill_betweenx(np.arange(y_lower, y_upper), 0, cluster_silhouettes,
                     facecolor=colors[i], edgecolor=colors[i], alpha=0.7)

    # Ajouter le label au milieu du cluster
    plt.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i), fontweight='bold')
    y_lower = y_upper + 10

plt.xlabel('Valeur de Silhouette')
plt.ylabel('Index des Clusters')
plt.title('Silhouettes Individuelles par Cluster', fontweight='bold')

# Ligne verticale pour la silhouette moyenne
avg_silhouette = silhouette_score(X_example_scaled, clusters_example)
plt.axvline(x=avg_silhouette, color="red", linestyle="--", linewidth=2,
           label=f'Moyenne: {avg_silhouette:.3f}')
plt.legend()

# Scatter plot avec silhouettes colorées
plt.subplot(2, 2, 2)
scatter = plt.scatter(X_example[:, 0], X_example[:, 1], c=silhouette_individual,
                     cmap='RdYlBu', alpha=0.7, s=50)
plt.colorbar(scatter, label='Silhouette individuelle')
plt.title('Données colorées par Silhouette', fontweight='bold')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')

# Histogramme des silhouettes
plt.subplot(2, 2, 3)
plt.hist(silhouette_individual, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
plt.axvline(avg_silhouette, color='red', linestyle='--', linewidth=2,
           label=f'Moyenne: {avg_silhouette:.3f}')
plt.xlabel('Valeur de Silhouette')
plt.ylabel('Nombre de Points')
plt.title('Distribution des Silhouettes', fontweight='bold')
plt.legend()

# Analyser les points problématiques
problematic_points = silhouette_individual < 0
n_problematic = problematic_points.sum()

plt.subplot(2, 2, 4)
if n_problematic > 0:
    plt.scatter(X_example[~problematic_points, 0], X_example[~problematic_points, 1],
               c=clusters_example[~problematic_points], cmap='tab10', alpha=0.7, s=50,
               label=f'Points OK ({(~problematic_points).sum()})')
    plt.scatter(X_example[problematic_points, 0], X_example[problematic_points, 1],
               c='red', marker='x', s=100, alpha=0.9,
               label=f'Points problématiques ({n_problematic})')
else:
    plt.scatter(X_example[:, 0], X_example[:, 1], c=clusters_example,
               cmap='tab10', alpha=0.7, s=50)

plt.title('Points Problématiques (Silhouette < 0)', fontweight='bold')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()

plt.tight_layout()
plt.show()

print(f"\nAnalyse des points problématiques:")
print("=" * 50)
print(f"Points avec silhouette négative: {n_problematic} ({n_problematic/len(X_example)*100:.1f}%)")

if n_problematic > 0:
    print(f"Interprétation:")
    print(f"  • Ces points sont plus proches d'autres clusters que du leur")
    print(f"  • Possible mauvaise assignation")
    print(f"  • Candidats pour réassignation ou suppression")

    # Analyser par cluster
    for cluster_id in range(4):
        cluster_mask = clusters_example == cluster_id
        cluster_problematic = problematic_points[cluster_mask].sum()
        cluster_total = cluster_mask.sum()

        if cluster_total > 0:
            pct_problematic = cluster_problematic / cluster_total * 100
            print(f"  Cluster {cluster_id}: {cluster_problematic}/{cluster_total} problématiques ({pct_problematic:.1f}%)")

# Comparer différents algorithmes de clustering
print(f"\nComparaison d'algorithmes sur le dataset 'Difficile':")
print("=" * 60)

X_test, y_test = datasets['Difficile (proches)']
scaler_test = StandardScaler()
X_test_scaled = scaler_test.fit_transform(X_test)

algorithms = {
    'K-Means (K=4)': KMeans(n_clusters=4, random_state=42, n_init=10),
    'K-Means (K=3)': KMeans(n_clusters=3, random_state=42, n_init=10),
    'K-Means (K=5)': KMeans(n_clusters=5, random_state=42, n_init=10),
    'DBSCAN (eps=0.5)': DBSCAN(eps=0.5, min_samples=5)
}

algorithm_results = {}

print(f"{'Algorithme':<20} {'Silhouette':<12} {'Calinski':<12} {'Davies-Bouldin':<15} {'N_Clusters'}")
print("-" * 75)

for algo_name, algo in algorithms.items():
    clusters_algo = algo.fit_predict(X_test_scaled)

    # Calculer le nombre de clusters (exclure le bruit pour DBSCAN)
    n_clusters_algo = len(set(clusters_algo)) - (1 if -1 in clusters_algo else 0)

    if n_clusters_algo > 1:
        # Pour les métriques, exclure le bruit
        if -1 in clusters_algo:
            mask_no_noise = clusters_algo != -1
            X_for_metrics = X_test_scaled[mask_no_noise]
            clusters_for_metrics = clusters_algo[mask_no_noise]
        else:
            X_for_metrics = X_test_scaled
            clusters_for_metrics = clusters_algo

        if len(set(clusters_for_metrics)) > 1:
            sil_algo = silhouette_score(X_for_metrics, clusters_for_metrics)
            cal_algo = calinski_harabasz_score(X_for_metrics, clusters_for_metrics)
            db_algo = davies_bouldin_score(X_for_metrics, clusters_for_metrics)
        else:
            sil_algo = cal_algo = db_algo = -999
    else:
        sil_algo = cal_algo = db_algo = -999

    algorithm_results[algo_name] = {
        'silhouette': sil_algo,
        'calinski': cal_algo,
        'davies_bouldin': db_algo,
        'n_clusters': n_clusters_algo,
        'clusters': clusters_algo
    }

    print(f"{algo_name:<20} {sil_algo:<12.3f} {cal_algo:<12.1f} {db_algo:<15.3f} {n_clusters_algo}")

# Identifier le meilleur algorithme
best_algorithm = max(algorithm_results.keys(),
                    key=lambda k: algorithm_results[k]['silhouette'] if algorithm_results[k]['silhouette'] > -999 else -1)

print(f"\nMeilleur algorithme: {best_algorithm}")
print(f"  Silhouette: {algorithm_results[best_algorithm]['silhouette']:.3f}")
print(f"  Clusters détectés: {algorithm_results[best_algorithm]['n_clusters']}")

# Guide de choix de métrique
print(f"\nGuide de choix de métrique d'évaluation:")
print("=" * 60)

print("UTILISEZ SILHOUETTE SCORE quand:")
print("  • Vous voulez une métrique intuitive et normalisée")
print("  • Vous comparez différents algorithmes")
print("  • Vous voulez détecter les points mal classés")
print("  • Standard de l'industrie")

print("\nUTILISEZ CALINSKI-HARABASZ quand:")
print("  • Vous avez des clusters de tailles très différentes")
print("  • Vous voulez favoriser la compacité")
print("  • Calcul plus rapide sur gros datasets")

print("\nUTILISEZ DAVIES-BOULDIN quand:")
print("  • Vous voulez pénaliser les clusters proches")
print("  • Vous préférez des clusters bien séparés")
print("  • Alternative au silhouette score")

# Métriques externes (quand on connaît la vérité)
print(f"\nMétriques externes (quand vérité terrain disponible):")
print("=" * 60)

from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, fowlkes_mallows_score

# Utiliser le meilleur clustering trouvé
best_clusters = algorithm_results[best_algorithm]['clusters']

# Calculer les métriques externes
ari = adjusted_rand_score(y_test, best_clusters)
nmi = normalized_mutual_info_score(y_test, best_clusters)
fmi = fowlkes_mallows_score(y_test, best_clusters)

print(f"Adjusted Rand Index (ARI): {ari:.3f}")
print(f"  → Mesure l'accord global entre clusterings")
print(f"  → 1.0 = accord parfait, 0.0 = accord aléatoire")

print(f"\nNormalized Mutual Information: {nmi:.3f}")
print(f"  → Mesure l'information partagée")
print(f"  → 1.0 = information parfaitement partagée")

print(f"\nFowlkes-Mallows Index: {fmi:.3f}")
print(f"  → Moyenne géométrique de précision et rappel")
print(f"  → 1.0 = clustering parfait")

# Recommandations finales
print(f"\nRecommandations pour évaluer vos clusterings:")
print("=" * 60)

print("1. COMMENCEZ par la visualisation")
print("   • Tracez vos données si possible (2D/3D)")
print("   • Vérifiez visuellement la cohérence")

print("\n2. UTILISEZ plusieurs métriques")
print("   • Silhouette score (standard)")
print("   • Plus une métrique complémentaire")
print("   • Cherchez la cohérence entre métriques")

print("\n3. VALIDEZ avec l'expertise métier")
print("   • Les clusters ont-ils du sens business ?")
print("   • Sont-ils exploitables opérationnellement ?")
print("   • Apportent-ils des insights nouveaux ?")

print("\n4. TESTEZ la stabilité")
print("   • Relancez l'algorithme plusieurs fois")
print("   • Testez sur des sous-échantillons")
print("   • Vérifiez la robustesse aux paramètres")

# Cas où les métriques sont contradictoires
print(f"\nQue faire si les métriques se contredisent ?")
print("=" * 50)
print("• Privilégiez le silhouette score (plus fiable)")
print("• Visualisez les résultats pour comprendre")
print("• Testez sur un sous-échantillon")
print("• Consultez l'expertise métier")
print("• Considérez que vos données n'ont peut-être pas de structure claire")

print(f"\n💡 Règle d'or:")
print("Une bonne évaluation combine métriques quantitatives,")
print("visualisation et validation métier. Aucune métrique")
print("seule ne peut garantir la qualité d'un clustering !")